In [1]:
import os
from pathlib import Path

In [2]:
doc_dir = "/home/admin1/Code/RAG_VLLM/doc_folder_1"

files = list(Path(doc_dir).glob("*")) 
print(files)

[PosixPath('/home/admin1/Code/RAG_VLLM/doc_folder_1/Huong-dan-doc-va-phan-tich-BCTC-NHANH.pdf')]


In [3]:
file = files[0]
print(file)

/home/admin1/Code/RAG_VLLM/doc_folder_1/Huong-dan-doc-va-phan-tich-BCTC-NHANH.pdf


# Basic usage


### Using DocumentConverter to process a document.

In [4]:

from docling.document_converter import DocumentConverter

In [5]:
source = file
converter = DocumentConverter()
doc = converter.convert(source).document

/home/admin1/miniconda3/envs/chatbot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
print(doc.export_to_markdown()[:50000])

<!-- image -->

## Hướng dẫn đọc và phân tích báo cáo tài chính (NHANH)

<!-- image -->

<!-- image -->

## Mục lục

| Báo cáo tài chính là gì?....................................................................................................................4                                                                            |
|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Bước #1: Xem ý kiến của Kiểm toán viên.....................................................................................4                                                                                             |
| Bước #2: Đọc hiểu Bảng cân đối kế toán.....................................................................................6                                                                                             |

In [13]:
import os
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    PdfPipelineOptions,
    PictureDescriptionApiOptions
)
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling_core.types.doc.document import ImageRefMode

def convert_with_image_annotation(input_doc_path):
    # Thay đổi API key cho Together AI hoặc provider khác
    api_key = os.environ.get("TOGETHER_API_KEY")  # hoặc API_KEY tương ứng
    model = "meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo"
    
    # Cấu hình cho Llama Vision model
    picture_desc_api_option = PictureDescriptionApiOptions(
        # URL cho Together AI (hoặc provider khác hỗ trợ Llama)
        url="https://api.together.xyz/v1/chat/completions",
        prompt="Describe this image in sentences in a single paragraph.",
        params=dict(
            model=model,
            max_tokens=512,  # Giới hạn tokens cho response
            temperature=0.1,  # Giảm temperature để có response ổn định hơn
        ),
        headers={
            "Authorization": "Bearer " + api_key,
            "Content-Type": "application/json",
        },
        timeout=120,  # Tăng timeout vì Llama model có thể chậm hơn
    )

    pipeline_options = PdfPipelineOptions(
        do_picture_description=True,
        picture_description_options=picture_desc_api_option,
        enable_remote_services=True,
        generate_picture_images=True,
        images_scale=2,
    )

    converter = DocumentConverter(
        format_options={InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)}
    )
    
    conv_res = converter.convert(source=input_doc_path)
    return conv_res


In [14]:
result = convert_with_image_annotation(file)

In [15]:
print(result.document.export_to_markdown(mark_annotations = True, include_annotations=True))

<!-- image -->

## Hướng dẫn đọc và phân tích báo cáo tài chính (NHANH)

<!--<annotation kind="description">-->The image presents a close-up view of a newspaper page, with the title "TIMES" prominently displayed in white text on a dark blue background at the top right corner. The page is predominantly white, featuring a mix of black and red text, as well as some green accents. A prominent graph is situated in the center of the page, accompanied by various lines of text that appear to be in German. In the bottom right corner, a blue logo bearing the text "govalue" is visible. The overall atmosphere of the image suggests that it is a stock photo or advertisement for a financial news source, likely intended to convey a sense of professionalism and authority in the field of finance.<!--<annotation/>-->

<!-- image -->

<!-- image -->

## Mục lục

| Báo cáo tài chính là gì?....................................................................................................................4  

### Save 


In [16]:
def export_function_md_with_image_ref(conv_res, output_path:str, replace_blank:str="_"):
    
    output_dir = Path(output_path)
    output_dir.mkdir(parents=True, exist_ok=True)
    doc_filename = conv_res.input.file.stem.replace(" ", replace_blank)

    # Save markdown with externally referenced pictures
    md_filename = output_dir / f"{doc_filename}-with-image-refs.md"
    conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.REFERENCED, include_annotations=True)

In [17]:
export_function_md_with_image_ref(result, "outputs")